In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chrome_options = Options()
chrome_options.add_argument("--headless") 
chrome_options.add_argument("--no-sandbox") 
chrome_options.add_argument("--disable-gpu") 
chrome_options.add_argument("--window-size=1920x1080")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

wait = WebDriverWait(driver, 10)

driver.get('https://shoppinglive.naver.com/categories/dc:3')

status = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/section[2]/div[4]/div[1]/a[1]/div/div[2]/div[1]/span/span[1]')))
    
if status.text == '라이브':
    element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/section[2]/div[4]/div[1]/a[2]')))
    url = element.get_attribute('href')


print(url)

driver.quit()



In [ ]:
import time
from selenium import webdriver
import pychrome
chrome_options = Options()
chrome_options.add_argument("--headless") 
chrome_options.add_argument("--no-sandbox") 
chrome_options.add_argument("--disable-gpu") 
chrome_options.add_argument("--window-size=1920x1080")
service = Service(ChromeDriverManager().install())

chrome_options.add_argument("--remote-debugging-port=7000")  # DevTools 포트 설정
driver = webdriver.Chrome(service=service, options=chrome_options)

def log_request(request):
    url = request.get('url')
    if url and url.endswith('.ts'):
        print(url)
        
def handle_network_event(**kwargs):
    log_request(kwargs.get('request'))

def main():
    driver.get("https://view.shoppinglive.naver.com/lives/1446735?fm=shoppinglive&sn=home&tr=lim&swipe=%7B%22referrerType%22%3A%22CATEGORY%22%2C%22pagerType%22%3A%22ALL%22%2C%22sortType%22%3A%22LATEST%22%2C%22referrerKey%22%3A%22dc%3A3%22%7D")
    
    browser = pychrome.Browser(url="http://localhost:7000")
    tab = browser.list_tab()[0]
    tab.start()

    tab.Network.requestWillBeSent = handle_network_event
    tab.Network.enable()

    try:
        while True:
            # 계속해서 네트워크 요청을 모니터링
            time.sleep(5)  # 5초 대기 후 다시 체크
    except:
        tab.stop()
        driver.quit()
        

if __name__ == "__main__":
    main()




In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options  # 추가
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC  # 추가
from webdriver_manager.chrome import ChromeDriverManager  # 추가

chrome_options = Options()
chrome_options.add_argument("--headless") 
chrome_options.add_argument("--no-sandbox") 
chrome_options.add_argument("--disable-gpu") 
chrome_options.add_argument("--window-size=1920x1080")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

wait = WebDriverWait(driver, 5)

driver.get('https://shoppinglive.naver.com/categories/dc:2')

# 요소가 로드될 때까지 기다리기 (XPath)
try:
    svg_element = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/section[2]/div[4]/div[1]/a[1]/div/div[2]/div[1]/span/span[1]')))
    
    # fill 속성 가져오기
    fill_value = svg_element.text

    # 출력
    print("Fill value:", fill_value)

except Exception as e:
    print("요소를 찾을 수 없습니다:", e)

# 드라이버 종료
driver.quit()




In [ ]:
import os
import time
import requests
import subprocess
from selenium import webdriver
import pychrome
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--headless") 
chrome_options.add_argument("--no-sandbox") 
chrome_options.add_argument("--disable-gpu") 
chrome_options.add_argument("--window-size=1920x1080")
service = Service(ChromeDriverManager().install())

chrome_options.add_argument("--remote-debugging-port=7000")  # DevTools 포트 설정
driver = webdriver.Chrome(service=service, options=chrome_options)

# 초기화
first_segment = True  # 첫 번째 ts 파일을 처리하기 위한 플래그
output_file = "output_video.mp4"  # 최종 mp4 파일

def download_ts_file(url, idx):
    # ts 파일을 다운로드하는 함수
    ts_filename = f"segment_{idx}.ts"
    response = requests.get(url)
    with open(ts_filename, 'wb') as f:
        f.write(response.content)
    print(f"Downloaded: {ts_filename}")
    return ts_filename

def convert_first_ts_to_mp4(ts_filename):
    # 첫 번째 ts 파일을 mp4로 변환
    subprocess.run([
        'ffmpeg', '-i', ts_filename, '-c', 'copy', output_file
    ])
    print(f"Created MP4: {output_file}")

def append_ts_to_mp4(ts_filename):
    # 기존 mp4에 새로운 ts 파일을 병합
    temp_output = "temp_output.mp4"
    
    # ffmpeg concat 명령어를 사용하여 mp4와 새로운 ts 파일을 병합
    subprocess.run([
        'ffmpeg', '-i', output_file, '-i', ts_filename, '-c', 'copy', '-bsf:a', 'aac_adtstoasc', temp_output
    ])
    
    # 기존 output_file을 새로 업데이트된 파일로 교체
    os.replace(temp_output, output_file)
    print(f"Updated MP4 with: {ts_filename}")

def log_request(request, idx):
    global first_segment
    url = request.get('url')
    if url and url.endswith('.ts'):
        ts_filename = download_ts_file(url, idx)
        
        if first_segment:
            # 첫 번째 세그먼트인 경우에는 mp4 파일로 변환
            convert_first_ts_to_mp4(ts_filename)
            first_segment = False
        else:
            # 두 번째부터는 기존 mp4 파일에 새로 받은 ts 파일을 병합
            append_ts_to_mp4(ts_filename)

def handle_network_event(**kwargs):
    global idx
    request = kwargs.get('request')
    if request:
        log_request(request, idx)
        idx += 1

def main():
    global idx
    idx = 0  # ts 파일을 구분하기 위한 인덱스
    
    driver.get("https://view.shoppinglive.naver.com/lives/1446735?fm=shoppinglive&sn=home&tr=lim&swipe=%7B%22referrerType%22%3A%22CATEGORY%22%2C%22pagerType%22%3A%22ALL%22%2C%22sortType%22%3A%22LATEST%22%2C%22referrerKey%22%3A%22dc%3A3%22%7D")
    
    browser = pychrome.Browser(url="http://localhost:7000")
    tab = browser.list_tab()[0]
    tab.start()

    tab.Network.requestWillBeSent = handle_network_event
    tab.Network.enable()

    try:
        while True:
            # 계속해서 네트워크 요청을 모니터링
            time.sleep(5)  # 5초 대기 후 다시 체크
    except KeyboardInterrupt:
        print("Stopping...")
        tab.stop()
        driver.quit()

if __name__ == "__main__":
    main()



In [1]:
import os
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime, timedelta
from webdriver_manager.chrome import ChromeDriverManager

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def get_heart_count(driver, wait):
    try:
        heart_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".LikeButton_number_IXgmK.__disable_double_tab")))
        heart_count_text = heart_element.text.replace(',', '')
        return int(heart_count_text)
    except ValueError:
        print("하트 수 집계중")
        return None
    except Exception as e:
        print(f'Error in get_heart_count: {e}')
        return None

def get_chat_count(driver):
    try:
        chat_elements = driver.find_elements(By.CSS_SELECTOR, 'div.Comment_wrap_wRrdF')
        return len(chat_elements)
    except Exception as e:
        print(f'Error in get_chat_count: {e}')
        return 0

def log_results_to_csv(log_file, elapsed_time, heart_count, heart_increase, chat_count_in_interval):
    try:
        with open(log_file, 'a', newline='', encoding='utf-8') as file:
            writer = csv.writer(file)
            writer.writerow([elapsed_time, heart_count, heart_increase, chat_count_in_interval])
    except Exception as e:
        print(f"Error writing to CSV file: {e}")

def main():
    url = 'https://view.shoppinglive.naver.com/lives/1451804?fm=shoppinglive&sn=home&tr=lim&swipe=%7B%22referrerType%22%3A%22CATEGORY%22%2C%22pagerType%22%3A%22ALL%22%2C%22sortType%22%3A%22LATEST%22%2C%22referrerKey%22%3A%22dc%3A3%22%7D'
    
    log_file = os.path.abspath("heart_and_chat_increase_log.csv")
    print(f"Log file will be saved to: {log_file}")
    
    # CSV 파일에 헤더 추가
    with open(log_file, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(["시간", "라이크수", "라이크증가", "채팅증가수"])
    
    total_duration = 60
    start_time = time.time()

    driver = setup_driver()
    driver.get(url)
    wait = WebDriverWait(driver, 15)

    previous_heart_count = None

    try:
        while (time.time() - start_time) < total_duration:
            heart_count = get_heart_count(driver, wait)
            if heart_count is None:
                continue

            elapsed_time = timedelta(seconds=int(time.time() - start_time))  # 경과 시간 계산
            initial_chat_count = get_chat_count(driver)

            final_chat_count = get_chat_count(driver)
            chat_count_in_interval = final_chat_count - initial_chat_count

            if previous_heart_count is not None:
                heart_increase = heart_count - previous_heart_count
                log_results_to_csv(log_file, elapsed_time, heart_count, heart_increase, chat_count_in_interval)

            previous_heart_count = heart_count

    except Exception as e:
        print(f"Error in main loop: {e}")

    finally:
        driver.quit()

if __name__ == "__main__":
    main()


Log file will be saved to: c:\Users\Admin\Desktop\I_live_A_commerce\heart_and_chat_increase_log.csv
